<a href="https://colab.research.google.com/github/madSENSE0107/100sofReview/blob/main/Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI astrology content

#Install All Dependencies

In [2]:
!pip install swisseph
!pip install google-generativeai
!pip install TTS
!pip install moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.5 MB/s eta 0:00:00


#Generate astrological chart

In [6]:
!pip install pyswisseph
import swisseph as swe

from datetime import datetime

# User input
date_str = "1998-09-12"
time_str = "16:45"
timezone_offset = 5.5  # IST
latitude = 22.5726
longitude = 88.3639

# Convert to Julian Day
dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
jd = swe.julday(dt.year, dt.month, dt.day, dt.hour + dt.minute/60 - timezone_offset)

# Get planet positions
planets = {
    "Sun": swe.SUN, "Moon": swe.MOON, "Mars": swe.MARS,
    "Mercury": swe.MERCURY, "Jupiter": swe.JUPITER,
    "Venus": swe.VENUS, "Saturn": swe.SATURN, "Rahu": swe.MEAN_NODE
}

planet_positions = {}
for name, pid in planets.items():
    result, _ = swe.calc_ut(jd, pid)
    lon, lat, dist = result[:3]
    sign = int(lon / 30)
    degree = lon % 30
    planet_positions[name] = f"{degree:.2f}° in sign {sign+1}"

print(planet_positions)


{'Sun': '19.49° in sign 6', 'Moon': '11.55° in sign 3', 'Mars': '14.44° in sign 5', 'Mercury': '7.72° in sign 6', 'Jupiter': '23.54° in sign 12', 'Venus': '7.00° in sign 6', 'Saturn': '2.97° in sign 2', 'Rahu': '0.25° in sign 6'}


# Interpret with Gemini

In [11]:
import google.generativeai as genai
import re

# API key securely entered
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

# Create prompt
chart_str = "\n".join([f"{planet}: {pos}" for planet, pos in planet_positions.items()])
prompt_text = f"""You are a compassionate Vedic astrologer. Based on the following birth chart, give:
1. Likely past regrets
2. Likely future fears
3. Emotional tendencies

Chart:
{chart_str}
"""

# Generate interpretation
response = model.generate_content(prompt_text)
raw_interpretation = response.text.strip()

# Clean special characters for TTS
def clean_for_tts(text):
    # Remove emojis and most non-ASCII symbols
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Remove odd punctuation or bullet points
    text = re.sub(r'[•–—●★☆→]', '', text)
    # Replace multiple spaces or newlines with single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

interpretation = clean_for_tts(raw_interpretation)
print(interpretation)

Namaste, Based on the provided birth chart, let's delve into the likely emotional landscape of this individual. Remember, this is a general interpretation, and a full reading requires considering the entire chart, including the nakshatras (lunar mansions) and divisional charts. This is intended to offer guidance and understanding, not definitive predictions. **1. Likely Past Regrets:** The placement of Saturn in the 2nd house (family, speech, values) suggests a potential for past regrets concerning family relationships or expressing oneself authentically. There might be a sense of having held back emotions or not communicated effectively, leading to strained relationships or feelings of isolation. The conjunction of several planets (Mercury, Venus, Rahu, Sun) in the 6th house (daily routine, health, enemies) might indicate regrets related to career choices, health issues stemming from neglecting self-care, or feeling overwhelmed by daily struggles. Theres a possibility of regret over m

#Generate TTS voiceover

In [2]:
from TTS.api import TTS

# Choose and load a model (English, fast & decent)
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=False)

# Save the voiceover
tts.tts_to_file(text=interpretation, file_path="voiceover.mp3")


KeyboardInterrupt: 